# Getting start with PyMongo

## Install the MongoDB python driver PyMongo

In [ ]:
!pip install  pymongo

## Import packages

In [ ]:
import os
import pymongo
from pymongo import DESCENDING, ASCENDING
from bson.json_util import dumps

## Connection

#### Before connecting to your Atlas cluster, make sure your loaded sample data into it. 

If you didn't, please follow this [tutorial](https://github.com/nosql-esigelec/inge3a_23/blob/main/tp_1/configurations.md#cr%C3%A9er-une-sandbox-ou-bac-%C3%A0-sable).<br>
After doing it, get your connection string and let's continue.

In [ ]:
# set username, password and host name for your Atlas cluster
# create ENV variables or set directly your username and password
username = os.getenv('USERNAME', 'stechos')
password = os.getenv('PASSWORD', 'stechos')
host = "serial-techos.hxiqgqy.mongodb.net"

`os.getenv` allows you to get environment variables that you set previously. <br>
The first argument is the name of the variable.<br>
The second argument is the default value to use if the variable doens't exist

In [ ]:
client = pymongo.MongoClient(f"mongodb+srv://{username}:{password}@{host}/?retryWrites=true&w=majority")

In [ ]:
client.stats

In [ ]:
# list the existing databases
client.list_database_names()

## CRUD: Create, Read, Update, Delete

## Create

#### Let's connect to database and check data

In [ ]:
product_db = client.productDB

In [ ]:
#print collection names
product_db.list_collection_names()

In [ ]:
#store the collection products into a variable
products = product_db.products

In [ ]:
#check one of the data here
products.find_one({})

#### Inserting data
Let's add a new product to our store called `Mjöllnir`. It cost `30`€ but don't received it yet so `we don't have any of it` for the moment.


In [ ]:
inserted_product = products.insert_one({"ProductName": "Mjöllir", "Price":30})

In [ ]:
#the attribute aknowleged let us know that the write has been applied in memory 
inserted_product.acknowledged

In [ ]:
#after inserting data, we also have an id returned of the object
inserted_product.inserted_id

In [ ]:
#with this id, I can retrieve the document that I just inserted
products.find_one({"_id": inserted_product.inserted_id})

#### Inserting data from a csv file

In [ ]:
import pandas as pd
import numpy as np
import datetime

Lets create a new collection to backup the initial products collection

In [ ]:
products_bup = product_db.products_backup
#make sure that the collection is empty
products_bup.drop()

In [ ]:
#read the csv file as a list of dictionnaries
product_df = pd.read_csv('mongo-data/products.csv', sep=",")
product_df['Tags'] = product_df['Tags'].apply(eval)
product_list = product_df.to_dict(orient="records")
product_list

In [ ]:
products_bup.insert_many(product_list)

In [ ]:
print(dumps(products_bup.find_one({}), indent=2))

## Update 

#### Updating data
We just received the products and we noticed that we put a wring name at the insertion of data. Let's update the document of this product.

In [ ]:
#create a document with values to update/add
mjollnir = {"ProductName": "Mjöllnir", "Quantity":100}

In [ ]:
update_result = products.update_one({"ProductName": "Mjöllir"},{"$set":mjollnir})
print(f"Number of documents: {products.count_documents({})}")
print(update_result.raw_result)

#### Upserting data
Upsert is inserting data if not filter is doesn't find a match.<br>
Let's take an example with a new product.

In [ ]:
#creating a document for stormbreaker
stormbreaker = {"ProductName": "Stormbreaker", "price":50, "Quantity":100}

In [ ]:
stormbreaker_update = products.update_one({"ProductName": "Stormbreaker"},{"$set":stormbreaker})
print(f"Number of documents: {products.count_documents({})}")
print(stormbreaker_update.raw_result)

The number of documents didn't changed from the last time.
Nothing happened in the update because this product doen't exist. In some case, we would like to insert the document if it doesn't exist. That's why we can use the attribute `upsert`.

In [ ]:
stormbreaker_update = products.update_one({"ProductName": "Stormbreaker"},{"$set":stormbreaker}, 
                                          upsert=True)
print(f"Number of documents: {products.count_documents({})}")
print(stormbreaker_update.raw_result)

The number of documents increased. In fact the non-existing document has been inserted.

#### Updating some informations on products

- **Numerical attribute**

Let's say we want to add increase the quantity of a product with 10 units

In [ ]:
product_filter = {"ProductName": "1 Gallon Soy Milk"}

In [ ]:
products_bup.update_one(product_filter, {"$inc": {"Quantity":10}})

In [ ]:
print(dumps(products_bup.find_one(product_filter), indent=2))

- **Array attribute** (add an item to an **existing** array)

We'll use the product "Red Wine for this example.

In [ ]:
red_wine = {"ProductName": "Red Wine"}

Let's add a tag on it 

In [ ]:
products_bup.update_one(red_wine, {"$push": {"Tags": "Happiness"}})

In [ ]:
print(dumps(products_bup.find_one(red_wine), indent=2))

- **Array attribute** (add an item to an **non-existing** array)

We would like to add warning about some products like alocohol 

In [ ]:
products_bup.update_one(red_wine, {"$push": {"Warnings": "Prohibited under 18"}})

In [ ]:
print(dumps(products_bup.find_one(red_wine), indent=2))

In [ ]:
#Let's say we would like to add an arrival date to all products
arrival_update = products_bup.update_many({}, {"$set":{"ArrivalDate": datetime.datetime.now()}})

In [ ]:
print(dumps(products_bup.find({}, {"_id":0,"ProductName":1, "ArrivalDate":1}), indent=2))

In [ ]:
print(f"Acknowledge: {arrival_update.acknowledged}")
print(f"Mached Count: {arrival_update.matched_count}")
print(f"Modified Count: {arrival_update.modified_count}")
print(f"Upserted Id: {arrival_update.upserted_id}")

## Delete 

Let's delete a specific product. 

In [ ]:
product_filter = {"ProductName": "Gatorade"}
print(f"Before Delete: The number of products is {len(list(products_bup.find(product_filter)))}")
products_bup.delete_one(product_filter)
print(f"After Delete: The number of products is {len(list(products_bup.find(product_filter)))}")

We'll use products that have less than 10 items for our example.

In [ ]:
product_to_delete = {"Quantity": {"$lte":10}}

Let's delete the first product matching this condition.<br>
We can use the attribute deleted_count to check the number of deleted items.

In [ ]:
print(f"Before Delete: The number of products is {len(list(products_bup.find(product_to_delete)))}")
delete_one_operation = products_bup.delete_one(product_to_delete)
print(f"After Delete: The number of products is {len(list(products_bup.find(product_to_delete)))}")
print(f"The number of documents deleted is {delete_one_operation.deleted_count}")

In [ ]:
print(f"Before Delete: The number of products is {len(list(products_bup.find(product_to_delete)))}")
delete_many_operation = products_bup.delete_many(product_to_delete)
print(f"After Delete: The number of products is {len(list(products_bup.find(product_to_delete)))}")
print(f"The number of documents deleted is {delete_many_operation.deleted_count}")

## Read

#### Let's connect to database and check data

In [ ]:
mflix_db = client.sample_mflix

In [ ]:
# list the existing collections in MFlix
mflix_db.list_collection_names()

In [ ]:
movies = mflix_db.movies

In [ ]:
movies.count_documents({})

### First operations

- Reading with find_one()
- Reading with find
    - Iterating through cursors
- Field projection and filtering

In [ ]:
# find one movie record
movies.find_one({})

In [ ]:
# find one movie record with Salma Hayek as an actor
movies.find_one({"cast": "Salma Hayek"})

In [ ]:
# display the movies in which Salma Hayek acted
salma = movies.find({"cast": "Salma Hayek"})
print(dumps(salma, indent=2))

In [ ]:
# display the titles of movies in which Salma Hayek acted
salma_movies_titles = movies.find(#to complete)
print(dumps(salma_movies_titles, indent=2))

In [ ]:
# display ONLY the titles of movies in which Salma Hayek acted
salma_movies = movies.find(#to complete)
print(dumps(salma_movies, indent=2))

### Operations on cusrsor and aggregations
Documentation for aggregation pipeline: 
- https://www.mongodb.com/docs/manual/core/aggregation-pipeline/
- https://pymongo.readthedocs.io/en/stable/examples/aggregation.html

#### Limiting

In [ ]:
limited = movies.find(
{"directors": "Sam Raimi"},
{"_id":0, "title": 1, "cast":1}
).limit(2)
print(dumps(limited, indent=2))

In [ ]:
pipeline = [
{"$match" : {"directors": "Sam Raimi"}},
{"$project": {"_id": 0, "title":1, "cast":1}},
{"$limit": 2}
]

In [ ]:
limited_agg = movies.aggregate(pipeline)
print(dumps(limited_agg, indent=2))

#### Sorting

In [ ]:
sorted_cursor = movies.find(
    {"directors": "Sam Raimi"},
    {"_id":0, "year":1, "title":1, "cast": 1}
).sort("year", ASCENDING)
print(dumps(sorted_cursor, indent=2))

In [ ]:
pipeline = [
{"$match" : {"directors": "Sam Raimi"}},
{"$project": {"_id": 0, "year":1, "title":1, "cast":1}},
{"$sort": {"year": DESCENDING}}
]

In [ ]:
sorted_agg = movies.aggregate(pipeline)
print(dumps(sorted_agg, indent=2))

##### List movies in which Tom Hanks acted sorted by year and title

In [ ]:
tom_hanks_cursor = movies.find()
print(dumps(sorted_cursor, indent=2))

In [ ]:
pipeline = []
print(dumps(sorted_agg, indent=2))

#### Skipping

In [ ]:
#count the number of documents in which Sam Raimi acted
pipeline = [
    {"$match": {"directors": "Sam Raimi"}},
    {"$count": "num_movies"}
]
aggregation = movies.aggregate(pipeline)
print(dumps(aggregation,indent=2))

##### Skip records and take only the last one

In [ ]:
skipped = movies.find({"directors": "Sam Raimi"},{"_id": 0, "title":1, "cast":1}).skip(12)
print(dumps(skipped, indent=2))

##### Skip records and take only the 3 last ones

In [ ]:
skipped_sorted = movies.find({"directors": "Sam Raimi"},{"_id": 0, "title":1, "cast":1}).sort("year").skip(10)
print(dumps(skipped_sorted, indent=2))

In [ ]:
pipeline = [
{"$match": {"directors": "Sam Raimi"}},
{"$project": {"_id":0, "year":1, "title":1,"cast":1}},
{"$sort": {"year": ASCENDING}},
{"$skip": 10}]

sorted_skipped_agg = movies.aggregate(pipeline)
print(dumps(sorted_skipped_agg, indent=2))